In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder\
    .appName("Day20")\
    .getOrCreate()

In [9]:
raw_drivers = [
("D001","Ramesh","35","Hyderabad","Car,Bike"),
("D002","Suresh","Forty","Bangalore","Auto"),
("D003","Anita",None,"Mumbai",["Car"]),
("D004","Kiran","29","Delhi","Car|Bike"),
("D005","", "42","Chennai",None)
]
raw_cities = [
("Hyderabad","South"),
("Bangalore","South"),
("Mumbai","West"),
("Delhi","North"),
("Chennai","South")
]
raw_trips = [
("T001","D001","Hyderabad","2024-01-05","Completed","450"),
("T002","D002","Bangalore","05/01/2024","Cancelled","0"),
("T003","D003","Mumbai","2024/01/06","Completed","620"),
("T004","D004","Delhi","invalid_date","Completed","540"),
("T005","D001","Hyderabad","2024-01-10","Completed","700"),
("T006","D005","Chennai","2024-01-12","Completed","350")
]
raw_activity = [
("D001","login,accept_trip,logout","{'device':'mobile'}",180),
("D002",["login","logout"],"device=laptop",60),
("D003","login|accept_trip",None,120),
("D004",None,"{'device':'tablet'}",90),
("D005","login","{'device':'mobile'}",30)
]


In [17]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType,DoubleType
drivers_schema=StructType([
    StructField("driver_id",StringType(),True),
    StructField("name",StringType(),True),
    StructField("age",StringType(),True),
    StructField("city",StringType(),True),
    StructField("vehicle_types",StringType(),True)
])

cities_schema=StructType([
    StructField("city",StringType(),True),
    StructField("region",StringType(),True)
])
trips_schema=StructType([
    StructField("trip_id",StringType(),True),
    StructField("driver_id",StringType(),True),
    StructField("city",StringType(),True),
    StructField("trip_date_raw",StringType(),True),
    StructField("trip_status",StringType(),True),
    StructField("fare_raw",StringType(),True)
])
activity_schema=StructType([
    StructField("driver_id",StringType(),True),
    StructField("actions",StringType(),True),
    StructField("metadata_raw", StringType(), True),
    StructField("session_duration_sec", IntegerType(), True),

])
drivers_df0 = spark.createDataFrame(raw_drivers, schema=drivers_schema)
cities_df   = spark.createDataFrame(raw_cities,  schema=cities_schema)
trips_df0   = spark.createDataFrame(raw_trips,   schema=trips_schema)
activity_df0= spark.createDataFrame(raw_activity,schema=activity_schema)

drivers_df0.show(truncate=False);
trips_df0.show(truncate=False);
activity_df0.show(truncate=False);
cities_df.show(truncate=False)

+---------+------+-----+---------+-------------+
|driver_id|name  |age  |city     |vehicle_types|
+---------+------+-----+---------+-------------+
|D001     |Ramesh|35   |Hyderabad|Car,Bike     |
|D002     |Suresh|Forty|Bangalore|Auto         |
|D003     |Anita |NULL |Mumbai   |[Car]        |
|D004     |Kiran |29   |Delhi    |Car|Bike     |
|D005     |      |42   |Chennai  |NULL         |
+---------+------+-----+---------+-------------+

+-------+---------+---------+-------------+-----------+--------+
|trip_id|driver_id|city     |trip_date_raw|trip_status|fare_raw|
+-------+---------+---------+-------------+-----------+--------+
|T001   |D001     |Hyderabad|2024-01-05   |Completed  |450     |
|T002   |D002     |Bangalore|05/01/2024   |Cancelled  |0       |
|T003   |D003     |Mumbai   |2024/01/06   |Completed  |620     |
|T004   |D004     |Delhi    |invalid_date |Completed  |540     |
|T005   |D001     |Hyderabad|2024-01-10   |Completed  |700     |
|T006   |D005     |Chennai  |2024-01-1

In [18]:
from pyspark.sql.functions import col
drivers_df0.show()
cities_df.show()
trips_df0.show()
activity_df0.show()

+---------+------+-----+---------+-------------+
|driver_id|  name|  age|     city|vehicle_types|
+---------+------+-----+---------+-------------+
|     D001|Ramesh|   35|Hyderabad|     Car,Bike|
|     D002|Suresh|Forty|Bangalore|         Auto|
|     D003| Anita| NULL|   Mumbai|        [Car]|
|     D004| Kiran|   29|    Delhi|     Car|Bike|
|     D005|      |   42|  Chennai|         NULL|
+---------+------+-----+---------+-------------+

+---------+------+
|     city|region|
+---------+------+
|Hyderabad| South|
|Bangalore| South|
|   Mumbai|  West|
|    Delhi| North|
|  Chennai| South|
+---------+------+

+-------+---------+---------+-------------+-----------+--------+
|trip_id|driver_id|     city|trip_date_raw|trip_status|fare_raw|
+-------+---------+---------+-------------+-----------+--------+
|   T001|     D001|Hyderabad|   2024-01-05|  Completed|     450|
|   T002|     D002|Bangalore|   05/01/2024|  Cancelled|       0|
|   T003|     D003|   Mumbai|   2024/01/06|  Completed|     6

In [21]:
from pyspark.sql.functions import col,when
drivers_df=drivers_df0.withColumn("age",when((col("age")>0)&(col("age")<=100),col("age")).otherwise(None))


In [23]:
trips_df=trips_df0.withColumn(
    "fare",
    when(col("fare")>=0,col("fare")).otherwise(None)
)

{"ts": "2025-12-22 11:45:30.317", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `fare` cannot be resolved. Did you mean one of the following? [`city`, `fare_raw`, `trip_id`, `driver_id`, `trip_status`]. SQLSTATE: 42703", "context": {"file": "line 3 in cell [23]", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o336.withColumn.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `fare` cannot be resolved. Did you mean one of the following? [`city`, `fare_raw`, `trip_id`, `driver_id`, `trip_status`]. SQLSTATE: 42703;\n'Project [trip_id#68, driver_id#69, city#70, trip_date_raw#71, trip_status#72, fare_raw#73, CASE WHEN '`>=`('fare, 0) THEN 'fare ELSE null END AS fare#207]\n+- LogicalRDD [tri

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `fare` cannot be resolved. Did you mean one of the following? [`city`, `fare_raw`, `trip_id`, `driver_id`, `trip_status`]. SQLSTATE: 42703;
'Project [trip_id#68, driver_id#69, city#70, trip_date_raw#71, trip_status#72, fare_raw#73, CASE WHEN '`>=`('fare, 0) THEN 'fare ELSE null END AS fare#207]
+- LogicalRDD [trip_id#68, driver_id#69, city#70, trip_date_raw#71, trip_status#72, fare_raw#73], false


In [26]:
from pyspark.sql.functions import to_date
trips_df=trips_df0.withColumn("trip_date",to_date(col("trip_date_raw"),"yyyy-MM-dd"))
trips_df.show()

DateTimeException: [CANNOT_PARSE_TIMESTAMP] Text '05/01/2024' could not be parsed at index 0. Use `try_to_timestamp` to tolerate invalid input string and return NULL instead. SQLSTATE: 22007

In [28]:
from pyspark.sql.functions import split
drivers_df=drivers_df0.withColumn("vehicle_types",split(col("vehicle_types"),","))
activity_df=activity_df0.withColumn("actions",split(col("actions"),","))
drivers_df.show()
activity_df.show()

+---------+------+-----+---------+-------------+
|driver_id|  name|  age|     city|vehicle_types|
+---------+------+-----+---------+-------------+
|     D001|Ramesh|   35|Hyderabad|  [Car, Bike]|
|     D002|Suresh|Forty|Bangalore|       [Auto]|
|     D003| Anita| NULL|   Mumbai|      [[Car]]|
|     D004| Kiran|   29|    Delhi|   [Car|Bike]|
|     D005|      |   42|  Chennai|         NULL|
+---------+------+-----+---------+-------------+

+---------+--------------------+-------------------+--------------------+
|driver_id|             actions|       metadata_raw|session_duration_sec|
+---------+--------------------+-------------------+--------------------+
|     D001|[login, accept_tr...|{'device':'mobile'}|                 180|
|     D002|  [[login,  logout]]|      device=laptop|                  60|
|     D003| [login|accept_trip]|               NULL|                 120|
|     D004|                NULL|{'device':'tablet'}|                  90|
|     D005|             [login]|{'device

In [29]:
drivers_df=drivers_df0.dropna(subset=["driver_id"])
drivers_df.show()

+---------+------+-----+---------+-------------+
|driver_id|  name|  age|     city|vehicle_types|
+---------+------+-----+---------+-------------+
|     D001|Ramesh|   35|Hyderabad|     Car,Bike|
|     D002|Suresh|Forty|Bangalore|         Auto|
|     D003| Anita| NULL|   Mumbai|        [Car]|
|     D004| Kiran|   29|    Delhi|     Car|Bike|
|     D005|      |   42|  Chennai|         NULL|
+---------+------+-----+---------+-------------+



In [30]:
drivers_df.printSchema()
drivers_df.show(truncate=False)

root
 |-- driver_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- city: string (nullable = true)
 |-- vehicle_types: string (nullable = true)

+---------+------+-----+---------+-------------+
|driver_id|name  |age  |city     |vehicle_types|
+---------+------+-----+---------+-------------+
|D001     |Ramesh|35   |Hyderabad|Car,Bike     |
|D002     |Suresh|Forty|Bangalore|Auto         |
|D003     |Anita |NULL |Mumbai   |[Car]        |
|D004     |Kiran |29   |Delhi    |Car|Bike     |
|D005     |      |42   |Chennai  |NULL         |
+---------+------+-----+---------+-------------+



Part-B

In [33]:
trips_drivers_df=trips_df0.join(
    drivers_df,
    on="driver_id",
    how="inner"
)
trips_drivers_df.show()

+---------+-------+---------+-------------+-----------+--------+------+-----+---------+-------------+
|driver_id|trip_id|     city|trip_date_raw|trip_status|fare_raw|  name|  age|     city|vehicle_types|
+---------+-------+---------+-------------+-----------+--------+------+-----+---------+-------------+
|     D001|   T001|Hyderabad|   2024-01-05|  Completed|     450|Ramesh|   35|Hyderabad|     Car,Bike|
|     D001|   T005|Hyderabad|   2024-01-10|  Completed|     700|Ramesh|   35|Hyderabad|     Car,Bike|
|     D002|   T002|Bangalore|   05/01/2024|  Cancelled|       0|Suresh|Forty|Bangalore|         Auto|
|     D003|   T003|   Mumbai|   2024/01/06|  Completed|     620| Anita| NULL|   Mumbai|        [Car]|
|     D004|   T004|    Delhi| invalid_date|  Completed|     540| Kiran|   29|    Delhi|     Car|Bike|
|     D005|   T006|  Chennai|   2024-01-12|  Completed|     350|      |   42|  Chennai|         NULL|
+---------+-------+---------+-------------+-----------+--------+------+-----+-----

In [34]:
trips_drivers_cities_df=trips_drivers_df.join(
    cities_df,
    on="city",
    how="inner"
)
trips_drivers_cities_df.show()

+---------+---------+-------+-------------+-----------+--------+------+-----+---------+-------------+------+
|     city|driver_id|trip_id|trip_date_raw|trip_status|fare_raw|  name|  age|     city|vehicle_types|region|
+---------+---------+-------+-------------+-----------+--------+------+-----+---------+-------------+------+
|Bangalore|     D002|   T002|   05/01/2024|  Cancelled|       0|Suresh|Forty|Bangalore|         Auto| South|
|Hyderabad|     D001|   T005|   2024-01-10|  Completed|     700|Ramesh|   35|Hyderabad|     Car,Bike| South|
|Hyderabad|     D001|   T001|   2024-01-05|  Completed|     450|Ramesh|   35|Hyderabad|     Car,Bike| South|
|  Chennai|     D005|   T006|   2024-01-12|  Completed|     350|      |   42|  Chennai|         NULL| South|
|   Mumbai|     D003|   T003|   2024/01/06|  Completed|     620| Anita| NULL|   Mumbai|        [Car]|  West|
|    Delhi|     D004|   T004| invalid_date|  Completed|     540| Kiran|   29|    Delhi|     Car|Bike| North|
+---------+--------

In [37]:
from pyspark.sql.functions import broadcast
joined_final = trips_drivers_df.join(broadcast(cities_df), on="city", how="left")
joined_final.show()

+---------+---------+-------+-------------+-----------+--------+------+-----+---------+-------------+------+
|     city|driver_id|trip_id|trip_date_raw|trip_status|fare_raw|  name|  age|     city|vehicle_types|region|
+---------+---------+-------+-------------+-----------+--------+------+-----+---------+-------------+------+
|Hyderabad|     D001|   T001|   2024-01-05|  Completed|     450|Ramesh|   35|Hyderabad|     Car,Bike| South|
|Hyderabad|     D001|   T005|   2024-01-10|  Completed|     700|Ramesh|   35|Hyderabad|     Car,Bike| South|
|Bangalore|     D002|   T002|   05/01/2024|  Cancelled|       0|Suresh|Forty|Bangalore|         Auto| South|
|   Mumbai|     D003|   T003|   2024/01/06|  Completed|     620| Anita| NULL|   Mumbai|        [Car]|  West|
|    Delhi|     D004|   T004| invalid_date|  Completed|     540| Kiran|   29|    Delhi|     Car|Bike| North|
|  Chennai|     D005|   T006|   2024-01-12|  Completed|     350|      |   42|  Chennai|         NULL| South|
+---------+--------

In [39]:
from pyspark.sql.functions import broadcast
broadcast_df = trips_drivers_df.join(broadcast(cities_df), on="city", how="inner")
broadcast_df.show()

+---------+---------+-------+-------------+-----------+--------+------+-----+---------+-------------+------+
|     city|driver_id|trip_id|trip_date_raw|trip_status|fare_raw|  name|  age|     city|vehicle_types|region|
+---------+---------+-------+-------------+-----------+--------+------+-----+---------+-------------+------+
|Hyderabad|     D001|   T001|   2024-01-05|  Completed|     450|Ramesh|   35|Hyderabad|     Car,Bike| South|
|Hyderabad|     D001|   T005|   2024-01-10|  Completed|     700|Ramesh|   35|Hyderabad|     Car,Bike| South|
|Bangalore|     D002|   T002|   05/01/2024|  Cancelled|       0|Suresh|Forty|Bangalore|         Auto| South|
|   Mumbai|     D003|   T003|   2024/01/06|  Completed|     620| Anita| NULL|   Mumbai|        [Car]|  West|
|    Delhi|     D004|   T004| invalid_date|  Completed|     540| Kiran|   29|    Delhi|     Car|Bike| North|
|  Chennai|     D005|   T006|   2024-01-12|  Completed|     350|      |   42|  Chennai|         NULL| South|
+---------+--------

In [ ]:
valid_trips_df=trips_df.join(
    drivers_df.select("driver_id"),
    on="driver_id",
    how="inner"
)
valid_trips_df.show()

PART-3

In [ ]:
from pyspark.sql.functions import explode,count
total_trips_per_city=trips_drivers_cities_df.groupBy("city")\
.agg(count("trip_id").alias("total_trips"))
total_trips_per_city.show()


In [ ]:
from pyspark.sql.functions import sum
total_revenue_per_city=trips_drivers_cities_df.groupBy("city")\
.agg(sum("fare").alias("total_revenue"))
total_revenue_per_city.show()

In [ ]:
from pyspark.sql.functions import avg
avg_fare_per_driver=trips_df.groupBy("driver_id")\
.agg(avg("fare").alias("avg_fare"))
avg_fare_per_driver.show()

In [ ]:
completed_trips_per_driver=trips_df.groupBy("driver_id")\
.agg(count)